# Simple MLP Code From Scratch

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

In [ ]:
def sigmoid(x):
    return  1 /( 1 + (math.e)**(-1 * x))

In [ ]:
def sigmoid_derivative(x):
    a = sigmoid(x)
    a = np.reshape(a,(-1,1))
    b = 1 - sigmoid(x)
    b = np.reshape(b,(-1,1))
    b = np.transpose(b)
    return np.diag(np.diag(np.matmul(a,b)))

In [ ]:
# initializing parameters
split_ratio = 0.7
eta = 0.3
epochs = 20

### Data Reading

In [ ]:
data = pd.read_excel('dataset.xlsx', header=None)
data = np.array(data)

### Data Pre-processing

In [ ]:
split_line_number = int(np.shape(data)[0] * split_ratio)

# find min and max of training data (use only train data to avoid leakage)
min = np.min(data[:split_line_number,])
max = np.max(data[:split_line_number,])

for i in range(np.shape(data)[0]):
    for j in range(np.shape(data)[1]):
        data[i,j] = (data[i,j] - min) / (max - min)

x_train      = data[:split_line_number,:4]
x_validation = data[split_line_number:,:4]
y_train      = data[:split_line_number,4]
y_validation = data[split_line_number:,4]

### Weight Initialization

In [ ]:
input_dimension = np.shape(x_train)[1]
l1_neurons = 5
l2_neurons = 1


w1 = np.random.uniform(low=-1,high=1,size=(input_dimension,l1_neurons))
w2 = np.random.uniform(low=-1,high=1,size=(l1_neurons,l2_neurons))

### Train Function

In [ ]:
def train(x_train, y_train, w1, w2):
    sqr_err_epoch_train = []
    output_train = []

    for j in range(np.shape(x_train)[0]):
        # feed-forward

        # layer 1
        net1 = np.matmul(x_train[j],w1)
        o1 = sigmoid(net1)
        o1 = np.reshape(o1,(-1,1))

        # layer 2
        net2 = np.matmul(np.transpose(o1),w2)
        o2 = net2

        output_train.append(o2[0])

        # error
        err = y_train[j] - o2[0]
        sqr_err_epoch_train.append(err**2)

        # back propagation
        f_derivative = sigmoid_derivative(net1)
        w2_f_derivative = np.matmul(f_derivative,w2)
        w2_f_derivative_x = np.matmul(w2_f_derivative,np.transpose(np.reshape(x_train[j],(-1,1))))
        w1 = np.subtract(w1 , np.transpose((eta * err * -1 * 1 * w2_f_derivative_x)))
        w2 = np.subtract(w2 , (eta * err * -1 * 1 * o1))

    return w1, w2, output_train, sqr_err_epoch_train

### Validation Function

In [ ]:
def validate(x_validation, y_validation, w1, w2):
    sqr_err_epoch_validation = []
    output_validation = []

    for j in range(np.shape(x_validation)[0]):
        # feed-forward

        # layer 1
        net1 = np.matmul(x_validation[j], w1)
        o1 = sigmoid(net1)
        o1 = np.reshape(o1, (-1, 1))

        # layer 2
        net2 = np.matmul(np.transpose(o1), w2)
        o2 = net2

        output_validation.append(o2[0])

        # error
        err = y_validation[j] - o2[0]
        sqr_err_epoch_validation.append(err ** 2)

    return output_validation, sqr_err_epoch_validation

## Training and Evaluating the Model

In [ ]:
MSE_train = []
MSE_validation = []

for i in range(epochs):

    # training
    w1, w2, output_train, sqr_err_epoch_train = train(x_train, y_train, w1, w2)

    mse_epoch_train = 0.5 * ((sum(sqr_err_epoch_train))/np.shape(x_train)[0])
    MSE_train.append(mse_epoch_train)

    # validation
    output_validation, sqr_err_epoch_validation = validate(x_validation, y_validation, w1, w2)

    mse_epoch_validation = 0.5 * ((sum(sqr_err_epoch_validation))/np.shape(x_validation)[0])
    MSE_validation.append(mse_epoch_validation)

    print(
        f"Epoch {i}: MSE Loss Train: {mse_epoch_train}, MSE Loss Validation: {mse_epoch_validation}")


# train
m_train , b_train = np.polyfit(y_train,output_train,1)

# validation
m_validation , b_validation = np.polyfit(y_validation, output_validation, 1)

# plots
fig, axs = plt.subplots(3, 2)
axs[0, 0].plot(MSE_train, 'b')
axs[0, 0].set_title('MSE Train')
axs[0, 1].plot(MSE_validation, 'r')
axs[0, 1].set_title('MSE Validation')

axs[1, 0].plot(y_train, 'b')
axs[1, 0].plot(output_train, 'r')
axs[1, 0].set_title('Output Train')
axs[1, 1].plot(y_validation, 'b')
axs[1, 1].plot(output_validation, 'r')
axs[1, 1].set_title('Output Validation')

axs[2, 0].plot(y_train, output_train, 'b*')
axs[2, 0].plot(y_train, m_train*y_train+b_train, 'r')
axs[2, 0].set_title('Regression Train')
axs[2, 1].plot(y_validation, output_validation, 'b*')
axs[2, 1].plot(y_validation, m_validation*y_validation+b_validation, 'r')
axs[2, 1].set_title('Regression Validation')

plt.show()

In [ ]:
# save the results plot on a file in current folder
plt.savefig('Results.jpg')